<a href="https://colab.research.google.com/github/elichen/karpathyGPT/blob/main/Adversarial_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

In [2]:
block_size = 256
n_layer = 4
n_head = 4
n_embed = 256
n_ffwd = n_embed * 4
dropout = 0.2
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
![[ -f input.txt ]] || wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [4]:
text = open("input.txt").read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size, "".join(chars)

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [7]:
class AttentionHead(nn.Module):
    def __init__(self, head_dim):
        super().__init__()
        self.query = nn.Linear(n_embed, head_dim)
        self.key = nn.Linear(n_embed, head_dim)
        self.value = nn.Linear(n_embed, head_dim)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        wei = q @ q.transpose(-2, -1) * k.shape[1] ** -0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        output = wei @ v
        return output

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_dim):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(head_dim) for _ in range(num_heads)])
        self.linear = nn.Linear(num_heads * head_dim, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.linear(out)
        out = self.dropout(out)
        return out

In [9]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
          nn.Linear(n_embed, n_ffwd),
          nn.ReLU(),
          nn.Linear(n_ffwd, n_embed),
          nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [10]:
class Block(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        self.sa_heads = MultiHeadAttention(num_heads, n_embed//num_heads)
        self.ffwd = FeedForward()
        self.norm1 = nn.LayerNorm(n_embed)
        self.norm2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_heads(self.norm1(x))
        x = x + self.ffwd(self.norm2(x))
        return x

In [11]:
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, n_embed)
        self.position_embeddings = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_head) for _ in range(n_layer)])
        self.norm = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, inputs):
        B,T = inputs.shape
        tok_emb = self.token_embeddings(inputs)
        pos_emb = self.position_embeddings(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.norm(x)
        return self.lm_head(x)

    def calculate_loss(self, inputs, targets):
        logits = self(inputs)
        return F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

    def generate(self, input_batch, max_new_tokens):
        for _ in range(max_new_tokens):
            logits = self(input_batch[:,-block_size:])
            last_logits = logits[:, -1, :]
            probabilities = F.softmax(last_logits, dim=-1)
            next_tokens = torch.multinomial(probabilities, 1).squeeze(1)
            input_batch = torch.cat((input_batch, next_tokens.unsqueeze(1)), dim=1)
        return input_batch

In [12]:
weights_file = 'tinyshakespeare.pt'

In [13]:
![[ -f {weights_file} ]] || wget -O {weights_file} https://github.com/elichen/karpathyGPT/raw/main/{weights_file}

--2024-02-24 19:25:02--  https://github.com/elichen/karpathyGPT/raw/main/tinyshakespeare.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/elichen/karpathyGPT/main/tinyshakespeare.pt [following]
--2024-02-24 19:25:03--  https://raw.githubusercontent.com/elichen/karpathyGPT/main/tinyshakespeare.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17286918 (16M) [application/octet-stream]
Saving to: ‘tinyshakespeare.pt’

tinyshakespeare.pt  100%[===================>]  16.49M  84.5MB/s    in 0.2s    

2024-02-24 19:25:04 (84.5 MB/s) - ‘tinyshakespeare.pt’ saved [17286918/17286918]



In [14]:
model = LanguageModel().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
model.load_state_dict(torch.load(weights_file, map_location=torch.device(device)))

3.258433 M parameters


<All keys matched successfully>

In [74]:
prefix = "First Citizen:"
context = torch.tensor(encode(prefix), dtype=torch.long).to(device).unsqueeze(0)
print(decode(model.generate(context, max_new_tokens=100)[0].tolist()))

First Citizen: imany; lok na isuslanes
Of:
Hof say more of my captoly, away as eye have, we I tell sy.
By, tull th


In [78]:
plotlosses = PlotLosses()

not_allowed_tokens = None if allow_non_ascii else get_nonascii_toks(tokenizer)
adv_suffix = adv_string_init

for i in range(num_steps):

    # Step 1. Encode user prompt (behavior + adv suffix) as tokens and return token ids.
    input_ids = suffix_manager.get_input_ids(adv_string=adv_suffix)
    input_ids = input_ids.to(device)

    # Step 2. Compute Coordinate Gradient
    coordinate_grad = token_gradients(model,
                    input_ids,
                    suffix_manager._control_slice,
                    suffix_manager._target_slice,
                    suffix_manager._loss_slice)

    # Step 3. Sample a batch of new tokens based on the coordinate gradient.
    # Notice that we only need the one that minimizes the loss.
    with torch.no_grad():

        # Step 3.1 Slice the input to locate the adversarial suffix.
        adv_suffix_tokens = input_ids[suffix_manager._control_slice].to(device)

        # Step 3.2 Randomly sample a batch of replacements.
        new_adv_suffix_toks = sample_control(adv_suffix_tokens,
                       coordinate_grad,
                       batch_size,
                       topk=topk,
                       temp=1,
                       not_allowed_tokens=not_allowed_tokens)

        # Step 3.3 This step ensures all adversarial candidates have the same number of tokens.
        # This step is necessary because tokenizers are not invertible
        # so Encode(Decode(tokens)) may produce a different tokenization.
        # We ensure the number of token remains to prevent the memory keeps growing and run into OOM.
        new_adv_suffix = get_filtered_cands(tokenizer,
                                            new_adv_suffix_toks,
                                            filter_cand=True,
                                            curr_control=adv_suffix)

        # Step 3.4 Compute loss on these candidates and take the argmin.
        logits, ids = get_logits(model=model,
                                 tokenizer=tokenizer,
                                 input_ids=input_ids,
                                 control_slice=suffix_manager._control_slice,
                                 test_controls=new_adv_suffix,
                                 return_ids=True)

        losses = target_loss(logits, ids, suffix_manager._target_slice)

        best_new_adv_suffix_id = losses.argmin()
        best_new_adv_suffix = new_adv_suffix[best_new_adv_suffix_id]

        current_loss = losses[best_new_adv_suffix_id]

        # Update the running adv_suffix with the best candidate
        adv_suffix = best_new_adv_suffix
        is_success = check_for_attack_success(model,
                                 tokenizer,
                                 suffix_manager.get_input_ids(adv_string=adv_suffix).to(device),
                                 suffix_manager._assistant_role_slice,
                                 test_prefixes)


    # Create a dynamic plot for the loss.
    plotlosses.update({'Loss': current_loss.detach().cpu().numpy()})
    plotlosses.send()

    print(f"\nPassed:{is_success}\nCurrent Suffix:{best_new_adv_suffix}", end='\r')

    # Notice that for the purpose of demo we stop immediately if we pass the checker but you are free to
    # comment this to keep the optimization running for longer (to get a lower loss).
    if is_success:
        break

    # (Optional) Clean up the cache.
    del coordinate_grad, adv_suffix_tokens ; gc.collect()
    torch.cuda.empty_cache()